In [ ]:
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q transformers datasets jiwer soundfile torchaudio pydub vosk==0.3.45
!pip install -q gTTS


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 7.4 MB/s eta 0:00:00


In [ ]:
# Install the GitHub CLI
!sudo apt-key adv --keyserver hkps://keyserver.ubuntu.com --recv-key C99B11F0CD218288D4989EBF118AE8FEEB5588F1
!sudo apt-add-repository https://apt.releases.hashicorp.com $(lsb_release -cs)
!sudo apt update
!sudo apt install gh -y

Executing: /tmp/apt-key-gpghome.5otDzPdiFy/gpg.1.sh --keyserver hkps://keyserver.ubuntu.com --recv-key C99B11F0CD218288D4989EBF118AE8FEEB5588F1
gpg: keyserver receive failed: No data
Repository: 'deb https://apt.releases.hashicorp.com jammy jammy'
Description:
Archive for codename: jammy components: jammy
More info: https://apt.releases.hashicorp.com
Adding repository.
Press [ENTER] to continue or Ctrl-c to cancel.
Adding deb entry to /etc/apt/sources.list.d/archive_uri-https_apt_releases_hashicorp_com-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/archive_uri-https_apt_releases_hashicorp_com-jammy.list
Get:1 https://apt.releases.hashicorp.com jammy InRelease [12.9 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Err:1 https://apt.releases.hashicorp.com jammy InRelease
  The following signat

In [ ]:
import os
from gtts import gTTS
from pydub import AudioSegment

os.makedirs("dataset/wavs", exist_ok=True)
os.makedirs("dataset/txt", exist_ok=True)

texts = [
 "Good morning team. Let's start the meeting and review last week's tasks.",
 "We need to increase the testing coverage by twenty percent before release.",
 "Deadline for the proposal is next Friday; assign owners and milestones.",
 "Please share the document after the meeting and update the tracker.",
 "Any blockers should be posted to the board and flagged for review."
]

for i, t in enumerate(texts, start=1):
    mp3_path = f"dataset/wavs/sample_{i}.mp3"
    wav_path = f"dataset/wavs/sample_{i}.wav"
    # create mp3 using gTTS
    gTTS(text=t, lang="en").save(mp3_path)
    # convert to 16kHz mono wav
    audio = AudioSegment.from_file(mp3_path)
    audio = audio.set_frame_rate(16000).set_channels(1)
    audio.export(wav_path, format="wav")
    # save ground truth
    with open(f"dataset/txt/sample_{i}.txt","w",encoding="utf-8") as f:
        f.write(t.lower())
print("✅ Synthetic dataset ready: 5 wavs + transcripts")


✅ Synthetic dataset ready: 5 wavs + transcripts


In [ ]:
import json, glob, wave
from vosk import Model, KaldiRecognizer

# Download Vosk model
model_dir = "vosk-model-small-en-us-0.15"
if not os.path.exists(model_dir):
    !wget -q https://alphacephei.com/vosk/models/vosk-model-small-en-us-0.15.zip -O /tmp/vosk.zip
    !unzip -q /tmp/vosk.zip -d .
    print("✅ Vosk model downloaded")

model = Model(model_dir)

def transcribe_vosk(path):
    wf = wave.open(path, "rb")
    rec = KaldiRecognizer(model, wf.getframerate())
    transcript = ""
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            j = json.loads(rec.Result())
            transcript += " " + j.get("text","")
    j = json.loads(rec.FinalResult())
    transcript += " " + j.get("text","")
    return transcript.strip().lower()

vosk_results = {}
for wav in sorted(glob.glob("dataset/wavs/*.wav")):
    vosk_results[wav] = transcribe_vosk(wav)
    print(wav, "->", vosk_results[wav])


✅ Vosk model downloaded
dataset/wavs/sample_1.wav -> good morning team let's start the meeting and review last week's tasks
dataset/wavs/sample_2.wav -> we need to increase the testing coverage by twenty percent before release
dataset/wavs/sample_3.wav -> deadline for the proposal is next friday a sign owners and milestones
dataset/wavs/sample_4.wav -> please share the document after the meeting and update the tracker
dataset/wavs/sample_5.wav -> any blockers should be posted to the board and flagged for review


In [ ]:
import whisper, glob
model = whisper.load_model("small")  # try "base" or "small" (avoid large on Colab CPU)

whisper_results = {}
for wav in sorted(glob.glob("dataset/wavs/*.wav")):
    r = model.transcribe(wav, fp16=False)
    whisper_results[wav] = r["text"].strip().lower()
    print(wav, "->", whisper_results[wav])


100%|███████████████████████████████████████| 461M/461M [00:08<00:00, 56.5MiB/s]


dataset/wavs/sample_1.wav -> good morning team, let's start the meeting and review last week's tasks
dataset/wavs/sample_2.wav -> we need to increase the testing coverage by 20% before release.
dataset/wavs/sample_3.wav -> deadline for the proposal is next friday. assign owners and milestones.
dataset/wavs/sample_4.wav -> please share the document after the meeting and update the tracker.
dataset/wavs/sample_5.wav -> any blockers should be posted to the board and flagged for review.


In [ ]:
from transformers import pipeline

asr = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-960h", chunk_length_s=10)

hf_results = {}
for wav in sorted(glob.glob("dataset/wavs/*.wav")):
    out = asr(wav)
    hf_results[wav] = out["text"].strip().lower()
    print(wav, "->", hf_results[wav])


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Device set to use cpu


dataset/wavs/sample_1.wav -> good morning team let's start the meeting in review last week's tasks
dataset/wavs/sample_2.wav -> we need to increase the testing coverage by twenty per cent before release
dataset/wavs/sample_3.wav -> dead line for the proposal is next friday assign owners and milestones
dataset/wavs/sample_4.wav -> please share the document after the meeting and update the tracker
dataset/wavs/sample_5.wav -> any blockers should be posted to the board and flagged for review


In [ ]:
from jiwer import wer

def load_gt(path):
    base = os.path.splitext(os.path.basename(path))[0]
    with open(f"dataset/txt/{base}.txt","r",encoding="utf-8") as f:
        return f.read().strip()

models = {"vosk": vosk_results, "whisper": whisper_results, "wav2vec2": hf_results}
report = {}

for name, res in models.items():
    scores = []
    for wav, hyp in res.items():
        ref = load_gt(wav)
        scores.append(wer(ref, hyp))
    avg = sum(scores)/len(scores)
    report[name] = round(avg,4)

print("📊 WER Report (lower = better):")
for k,v in report.items():
    print(f"  {k}: {v*100:.2f}%")


📊 WER Report (lower = better):
  vosk: 15.76%
  whisper: 8.48%
  wav2vec2: 20.76%


In [ ]:
import csv

with open("stt_benchmark_results.csv","w",newline="",encoding="utf-8") as csvfile:
    w = csv.writer(csvfile)
    w.writerow(["wav","ground_truth","vosk","whisper","wav2vec2"])
    for wav in sorted(glob.glob("dataset/wavs/*.wav")):
        ref = load_gt(wav)
        w.writerow([wav, ref, vosk_results[wav], whisper_results[wav], hf_results[wav]])

print("✅ Results saved to stt_benchmark_results.csv")


✅ Results saved to stt_benchmark_results.csv
